# Programming for Data Science and Artificial Intelligence

## 16. Convolutional Neural Networks from Scratch

### Readings

- [WEIDMAN] Ch6
- [CHARU] Ch8



In [1]:
#import from last time work so we can extend further
from neuralnet.second_version import *

So far we have focus on **Dense** layers (or also known as fully-connected layer) which are nice in understanding relationships.  Adding **activation function** like Sigmoid or Tanh allows us to understand the non-linear relationship between features and output, with Tanh having a steeper gradient, allowing the network to learn faster.  Adding **SoftMaxCrossEntropy** also enhance the gradient produced but remember that it only works with classification problems.  Adding **Dropout** helps in overfitting; **glorot initialization** to make sure the weight is normally distributed, **learning decay** to make sure we eventually reach the minimum instead of hopping all over the places, and last, the **momentum** to make sure we do not stuck in local minimum.  Such architecture is usally quite okay for **normal classification** problem.   

However, when we talk about specific classification problem such as image or text or signal, they all have specific nature that would benefit from different architectures.   

Today, we gonna work on image (this field is called computer vision) and discuss why Dense layer may not be the best, and propose CNN (Convolutional Layer) as a better way for dealing with image classification.

### Convolutional Layer
Let's say given a image of 24 x 24 pixels = 576 features.  We might input these features into Dense layers and try to ask the Dense layers to understand the relationships.

However, this is not so optimal since we do not **actually understand the nature of image**.  The key is that each single pixel actually holds very little information, right?  However, pattern of image can be better recognized by patches of pixels, rather than single pixel.  Imagine I give you a picture of cat, and I give you only a one-fourth of the picture, can you recognize that it's a cat?  Probably yes.  But what if I give you a single pixel.....you will have zero idea. 

**Why pattern of images are better recognized by patches?**...because humans recognize some visual patterns like corners, edges, sharpness.  Combining all these visual patterns form the image.  This is how humans visualize, and in fact, we should also apply these principles to neural networks

**So how do we generate each patch of feature?**...actually, it is very easy.  We simply perform a convolution operation like this:

![](figures/no_padding_no_strides.gif)

Mathematically, it looks like this:

Let's say we have a 5 x 5 input image I:

$$ I = \begin{bmatrix}
i_{11} & i_{12} & i_{13} & i_{14} & i_{15}
\\
i_{21} & i_{22} & i_{23} & i_{24} & i_{25}
\\
i_{31} & i_{32} & i_{33} & i_{34} & i_{35}
\\
i_{41} & i_{42} & i_{43} & i_{44} & i_{45}
\\
i_{51} & i_{52} & i_{53} & i_{54} & i_{55}
\end{bmatrix}
$$

Each of this pixel may represent the brightness ranging from 0 to 255.

If we define a 3 x 3 patch which we commonly called **weights (W)** or in computer vision, we called **filters/kernels** like this:

$$ w = \begin{bmatrix}
w_{11} & w_{12} & w_{13}
\\
w_{21} & w_{22} & w_{23}
\\
w_{31} & w_{32} & w_{33}
\end{bmatrix}
$$